In [1]:
# Imports
import math

In [2]:
input_repr_dict = catalog.load('input_repr_dict')

[07/10/24 11:55:23] INFO     Loading data from input_repr_dict (PickleDataset)...               ]8;id=358071;file://C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=538321;file://C:\Users\omid\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

### Unpack all the dictionary keys

In [3]:
J = input_repr_dict["J"]
M = input_repr_dict["M"]
compat = input_repr_dict["compat"]
dur = input_repr_dict["dur"]
due = input_repr_dict["due"]
part_id = input_repr_dict["part_id"]

In [4]:
J


[
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 4, 5, 6, 7],
    [1, 2, 3, 6, 7],
    [1, 2, 3, 6, 7],
    [1, 2, 3, 6, 7],
    [1, 2, 3, 6, 7],
    [1, 2, 3, 6, 7],
    [1, 2, 3, 6, 7]
]

In [5]:
J_upd = [[i - 1 for i in sub_list] for sub_list in J]

In [6]:
J_upd


[
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 3, 4, 5, 6],
    [0, 1, 2, 5, 6],
    [0, 1, 2, 5, 6],
    [0, 1, 2, 5, 6],
    [0, 1, 2, 5, 6],
    [0, 1, 2, 5, 6],
    [0, 1, 2, 5, 6]
]

Tasks of each job

In [7]:
J[0]

[1, 2, 3, 4, 5, 6, 7]

List of machines M

In [8]:
M

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

Compatability matrix for a taks of a job

In [9]:
compat


[
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [15], [16, 17]],
    [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10], [11], [12, 13, 14], [7, 8, 9, 10], [1

Duration of a task of a job

In [10]:
dur[0][0]

162.0

Due date (max completion time) for a job

In [11]:
# Divide by 15 to indicate the number of 15 minute blocks that are required for the task
due = [math.ceil(due_date/15) for due_date in due]

In [12]:
due[0]

160

Task to machines mapping:

In [13]:
task_to_machines = {
  0: [1, 2, 3, 4, 5, 6],  # HAAS
  1: [7, 8, 9, 10],  # Inspection
  2: [11],  # First Wash
  3: [12, 13, 14],  # Manual Prep
  4: [7, 8, 9, 10],  # Inspection (again)
  5: [15],  # Final Wash
  6: [16, 17]  # Final inspection
}

In [14]:
dur


[
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8, 42.0, 54.0, 15.0, 25.2, 48.0],
    [162.0, 19.8,

In [15]:
# Divided by fifteen to express the number of 15 minute blocks are required for the task
dur_rounded_up = [[math.ceil(x/15) for x in sub_list] for sub_list in dur]

In [16]:
dur_rounded_up


[
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [28, 2, 4, 2, 4],
    [24, 2, 4, 2, 4],
    [24, 2, 4, 2, 4],
    [24, 2, 4, 2, 4],
    [24, 2, 4, 2, 4],
    [28, 2, 4, 2, 4]
]

In [17]:
# Iterate over the sublists to round up the duration
# If there are only five tasks in the job, pad them with 0s
for sublist in dur_rounded_up:
    # If the sublist has a length of 5
    if len(sublist) == 5:
        # Insert two zeros before the last two entries
        sublist.insert(-2, 0)
        sublist.insert(-2, 0)

In [18]:
dur_rounded_up


[
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [11, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [10, 2, 3, 4, 1, 2, 4],
    [28, 2, 4, 0, 0, 2, 4],
    [24, 2, 4, 0, 0, 2, 4],
    [24, 2, 4, 0, 0, 2, 4],
    [24, 2, 4, 0, 0, 2, 4],
    [24, 2, 4, 0, 0, 2, 4],
    [28, 2, 4, 0, 0, 2, 4]
]

In [19]:
task_to_machine_list = list(task_to_machines.values())

### Define the linear program

In [20]:
# Example parameters
T = range(1, 337)  # Time horizon: 1 to 336
J = ['batch1', 'batch2']  # Example batch set
K = { 'batch1': [1, 2, 3], 'batch2': [1, 3, 2] }  # Example task sequences
M = { 1: ['machine1'], 2: ['machine2'], 3: ['machine3'] }  # Example machine sets
d = { ('batch1', 1): 2, ('batch1', 2): 3, ('batch1', 3): 2,
      ('batch2', 1): 4, ('batch2', 3): 1, ('batch2', 2): 3 }  # Example durations
f = { 'batch1': 100, 'batch2': 150 }  # Example promised delivery times

In [22]:
import pulp

def linear_program(T, J, K, M, d, f):
    # Variables
    y = pulp.LpVariable.dicts("y", ((j, k, m, t) for j in J for k in K[j] for m in M[k] for t in T), 0, 1, pulp.LpBinary)
    x = pulp.LpVariable.dicts("x", ((j, k, m, t) for j in J for k in K[j] for m in M[k] for t in T), 0, 1, pulp.LpBinary)

    # Problem definition
    prob = pulp.LpProblem("SchedulingProblem", pulp.LpMaximize)
    
    # -- Objective -- 
    # Objective function: Minimize the starting times of tasks
    prob += pulp.lpSum(f[j] - (t * x[j, K[j][-1], m, t]) for j in J for m in M[K[j][-1]] for t in T)
                        
    # -- Constraints -- 
    # Each task in each batch starts once and only on one machine
    for j in J:
        for k in K[j]:
            prob += pulp.lpSum(y[j, k, m, t] for m in M[k] for t in T) == 1
     
    # A task only occupies a machine for the duration of processing time if the task is set to start on said machine
    for j in J:
        for k in K[j]:
            for m in M[k]:
                for t in T:
                    if t + d[j, k] - 1 <= max(T):
                        # Ensuring x values correspond correctly with the y start times
                        for t_prime in range(t, t + d[j, k]):
                            prob += x[j, k, m, t_prime] >= y[j, k, m, t_prime]

    # Total time the task occupies the machines equals the processing duration
    for j in J:
        for k in K[j]:
            for m in M[k]:
                prob += pulp.lpSum(x[j, k, m, t] for t in T) == d[j, k]
                
    # Only one machine can be used at a time
    for t in T:
        for k, m in M.items():
            mach = m[0]
            prob += pulp.lpSum(x[j, k, mach, t] for j in J) <= 1

    # Tasks must be done in the order specified by the batch
    for j in J:
        for idx in range(len(K[j]) - 1):
            k1 = K[j][idx]
            k2 = K[j][idx + 1]
            for m1 in M[k1]:
                for m2 in M[k2]:
                    prob += (pulp.lpSum((t + d[j, k1]) * y[j, k1, m1, t] for t in T) <= 
                             pulp.lpSum(t * y[j, k2, m2, t] for t in T))
                    
    # Tasks cannot be stopped midway, but have to be completed once they start
    for j in J:
        for k in K[j]:
            for m in M[k]:
                for t in T:
                    if t + d[j, k] - 1 <= max(T):
                        for t_prime in range(t, t + d[j, k]):
                            prob += x[j, k, m, t_prime] <= pulp.lpSum(y[j, k, m, t_prime_start] for t_prime_start in range(max(1, t_prime - d[j, k] + 1), t_prime + 1))
        
    # Solve the problem
    prob.solve()
    
    print("Status:", pulp.LpStatus[prob.status])
    
    # Print the optimal value of the objective function
    print("Optimal Value:", pulp.value(prob.objective))

    # Print the results
    for v in prob.variables():
        if v.varValue > 0.01:  #  and v.name.startswith('x')
            print(f"{v.name} = {v.varValue}")

            
    return x, y

# Example parameters
T = range(1, 337)  # Time horizon: 1 to 336
J = ['batch1', 'batch2']  # Example batch set
K = { 'batch1': [1, 2, 3], 'batch2': [1, 2, 3] }  # Example task sequences
M = { 1: ['machine1'], 2: ['machine2'], 3: ['machine3'] }  # Example machine sets
d = { ('batch1', 1): 2, ('batch1', 2): 3, ('batch1', 3): 4,
      ('batch2', 1): 2, ('batch2', 2): 3, ('batch2', 3): 4 }  # Example durations
f = { 'batch1': 100, 'batch2': 150 }  # Example promised delivery times

# Call the function
x, y = linear_program(T, J, K, M, d, f)


Status: Optimal
Optimal Value: 83924.0
x_('batch1',_1,_'machine1',_4) = 1.0
x_('batch1',_1,_'machine1',_5) = 1.0
x_('batch1',_2,_'machine2',_6) = 1.0
x_('batch1',_2,_'machine2',_7) = 1.0
x_('batch1',_2,_'machine2',_8) = 1.0
x_('batch1',_3,_'machine3',_10) = 1.0
x_('batch1',_3,_'machine3',_11) = 1.0
x_('batch1',_3,_'machine3',_12) = 1.0
x_('batch1',_3,_'machine3',_13) = 1.0
x_('batch2',_1,_'machine1',_1) = 1.0
x_('batch2',_1,_'machine1',_2) = 1.0
x_('batch2',_2,_'machine2',_3) = 1.0
x_('batch2',_2,_'machine2',_4) = 1.0
x_('batch2',_2,_'machine2',_5) = 1.0
x_('batch2',_3,_'machine3',_6) = 1.0
x_('batch2',_3,_'machine3',_7) = 1.0
x_('batch2',_3,_'machine3',_8) = 1.0
x_('batch2',_3,_'machine3',_9) = 1.0
y_('batch1',_1,_'machine1',_4) = 1.0
y_('batch1',_2,_'machine2',_6) = 1.0
y_('batch1',_3,_'machine3',_10) = 1.0
y_('batch2',_1,_'machine1',_1) = 1.0
y_('batch2',_2,_'machine2',_3) = 1.0
y_('batch2',_3,_'machine3',_6) = 1.0
